In [1]:
# ==========================================
# File: project_rdkt/notebooks/03_final_training_xgb.py
# ==========================================
# [RDKit 专属] XGBoost 最终训练 (V3: 稳健策略版)
# 策略调整:
# 之前 "Strict" (Depth=2) 欠拟合，"Balanced" (Depth=4) 效果变差。
# 现在切换到 "Robust" 模式:
# 1. Depth=3 (化学数据的黄金深度)
# 2. LR=0.01 (慢工出细活)
# 3. ColSample=0.5 (增加随机性，强迫模型多角度观察)

import os
import json
import warnings
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
# from xgboost.callback import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error

# ----------------------------------------------------------
# 0) 环境定位
# ----------------------------------------------------------
if 'notebooks' in os.getcwd():
    os.chdir('..')
print(f"🚀 [03] XGB Final Training (Robust Mode) | cwd = {os.getcwd()}")

# ----------------------------------------------------------
# 1) 数据读取
# ----------------------------------------------------------
try:
    df_train = pd.read_csv('data/processed/train.csv')
    df_val   = pd.read_csv('data/processed/val.csv')
    df_test  = pd.read_csv('data/processed/test.csv')
except FileNotFoundError:
    raise FileNotFoundError("❌ Missing data files. Run 01 first.")

target_col = 'Target_Log1o2' if 'Target_Log1o2' in df_train.columns else 'Target'

def get_Xy(df: pd.DataFrame):
    y = df[target_col]
    X = df.drop(columns=[target_col, 'SMILES_Meta', 'SMILES', 'ID'], errors='ignore')
    X = X.select_dtypes(include=[np.number])
    return X, y

X_train, y_train = get_Xy(df_train)
X_val,   y_val   = get_Xy(df_val)
X_test,  y_test  = get_Xy(df_test)

feat_cols = X_train.columns.tolist()
X_val = X_val.reindex(columns=feat_cols).fillna(0)
X_test = X_test.reindex(columns=feat_cols).fillna(0)

print(f"   Train: {X_train.shape} | Val: {X_val.shape} | Test: {X_test.shape}")

# ----------------------------------------------------------
# 2) 参数策略选择
# ----------------------------------------------------------
# STRATEGY = "strict"    # Depth=2, High Reg (欠拟合风险)
# STRATEGY = "balanced"  # Depth=4, Mid Reg (可能过拟合)
STRATEGY = "robust"      # Depth=3, Low LR (稳扎稳打)

print(f"\n🔧 当前策略: 【{STRATEGY.upper()}】")

if STRATEGY == "strict":
    params = {
        'objective': 'reg:squarederror', 'n_jobs': -1, 'random_state': 42,
        'max_depth': 2, 'learning_rate': 0.02, 'n_estimators': 5000,
        'min_child_weight': 6, 'gamma': 0.5,
        'subsample': 0.6, 'colsample_bytree': 0.6,
        'reg_alpha': 2.0, 'reg_lambda': 5.0,
        'eval_metric': 'rmse'
    }
elif STRATEGY == "balanced":
    params = {
        'objective': 'reg:squarederror', 'n_jobs': -1, 'random_state': 42,
        'max_depth': 4, 'learning_rate': 0.05, 'n_estimators': 5000,
        'min_child_weight': 3, 'gamma': 0.1,
        'subsample': 0.8, 'colsample_bytree': 0.8,
        'reg_alpha': 0.1, 'reg_lambda': 1.0,
        'eval_metric': 'rmse'
    }
else: # robust (Default for QSAR/Chemistry)
    params = {
        'objective': 'reg:squarederror',
        'n_jobs': -1,
        'random_state': 42,

        # --- 黄金参数区 ---
        'max_depth': 3,            # 经典深度
        'learning_rate': 0.01,     # 非常慢的学习率，配合早停
        'n_estimators': 10000,     # 给足空间让它慢慢跑

        # --- 强随机性 (关键) ---
        'subsample': 0.7,          # 每次只看 70% 数据
        'colsample_bytree': 0.5,   # 每次只看 50% 特征 (防止过度依赖单一特征)

        # --- 适度正则 ---
        'min_child_weight': 5,     # 稍微提高门槛，过滤噪声
        'gamma': 0.2,
        'reg_alpha': 1.0,          # L1
        'reg_lambda': 2.0,         # L2

        'eval_metric': 'rmse'
    }

print("\n🧠 Final Params:")
print(json.dumps({k: params[k] for k in ['max_depth', 'learning_rate', 'colsample_bytree', 'reg_alpha']}, indent=2))

# ----------------------------------------------------------
# 3) 训练 (Train)
# ----------------------------------------------------------
print("\n>>> Training with EarlyStopping on VAL ...")

model = XGBRegressor(**params, early_stopping_rounds=50)

model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=500
)

# 安全获取最佳轮次
try:
    best_iter = model.best_iteration
except AttributeError:
    try: best_iter = model.get_booster().best_iteration
    except: best_iter = "Unknown"

print(f"\n✅ Done. Best Iteration: {best_iter}")

# ----------------------------------------------------------
# 4) 预测 & 评估
# ----------------------------------------------------------
y_pred_train = model.predict(X_train)
y_pred_val   = model.predict(X_val)
y_pred_test  = model.predict(X_test)

def calc_r2(y, p): return r2_score(y, p)

r2_tr = calc_r2(y_train, y_pred_train)
r2_va = calc_r2(y_val, y_pred_val)
r2_te = calc_r2(y_test, y_pred_test)

print("\n" + "="*40)
print(f"📊 Performance ({STRATEGY.title()} Mode)")
print(f"   Train: {r2_tr:.4f}")
print(f"   Val  : {r2_va:.4f}")
print(f"   Test : {r2_te:.4f}")
print("="*40)

# ----------------------------------------------------------
# 5) 诊断: 特征重要性 (新增功能)
# ----------------------------------------------------------
# 如果效果还是很差，看看模型到底在学什么
try:
    importance = model.feature_importances_
    feats = X_train.columns
    df_imp = pd.DataFrame({'Feature': feats, 'Importance': importance}).sort_values('Importance', ascending=False)
    print("\n🧐 Top 10 Features Used by Model:")
    print(df_imp.head(10).to_string(index=False))
except:
    print("\n⚠️ 无法打印特征重要性")

# ----------------------------------------------------------
# 6) 保存结果
# ----------------------------------------------------------
os.makedirs('results/predictions', exist_ok=True)
pd.DataFrame({target_col: y_train.values, 'Predicted': y_pred_train}).to_csv('results/predictions/train_predictions.csv', index=False)
pd.DataFrame({target_col: y_val.values, 'Predicted': y_pred_val}).to_csv('results/predictions/val_predictions.csv', index=False)
pd.DataFrame({target_col: y_test.values, 'Predicted': y_pred_test}).to_csv('results/predictions/final_predictions.csv', index=False)

print("\n💾 预测结果已保存。")

🚀 [03] XGB Final Training (Robust Mode) | cwd = D:\HO\my_smiles_project\y_g.baby\project_rdkt
   Train: (105, 126) | Val: (23, 126) | Test: (23, 126)

🔧 当前策略: 【ROBUST】

🧠 Final Params:
{
  "max_depth": 3,
  "learning_rate": 0.01,
  "colsample_bytree": 0.5,
  "reg_alpha": 1.0
}

>>> Training with EarlyStopping on VAL ...
[0]	validation_0-rmse:1.64639	validation_1-rmse:1.63380


[500]	validation_0-rmse:0.50312	validation_1-rmse:0.97196


[787]	validation_0-rmse:0.37237	validation_1-rmse:0.95610



✅ Done. Best Iteration: 737

📊 Performance (Robust Mode)
   Train: 0.9453
   Val  : 0.6618
   Test : 0.6375

🧐 Top 10 Features Used by Model:
                Feature  Importance
       NumAromaticRings    0.052235
              PEOE_VSA1    0.051602
              PEOE_VSA6    0.040516
NumAromaticHeterocycles    0.036689
       MinPartialCharge    0.035000
              RingCount    0.030044
               fr_Al_OH    0.028326
           FractionCSP3    0.026873
                    Ipc    0.023033
                 AvgIpc    0.022634

💾 预测结果已保存。
